In [ ]:
import scanpy as sc
import scvelo as sv
import loompy as lp
import pandas as pd
%pylab inline

In [ ]:
sv.settings.figdir = 'figs_cell_report/'

# Preprocessing

In [ ]:
adata = sv.read_loom('6_files.loom')
adata.var_names_make_unique()

In [ ]:
adata.obs_names = [s.split(':')[1].split('.')[0] for s in adata.obs_names]

In [ ]:
# run this cell to ignore results from the first experiment
adata = adata[invert(adata.obs_names.str.startswith( ('A172', 'A257') ) ),:]
adata

In [ ]:
# Loads the corrected reads file
gene_scores = pd.read_csv('../../data/aggregated_1964_log.txt_filtered.txt', sep='\t', index_col=0).T

In [ ]:
# Manually corrects A267T1 -> A267T01
def correct(s):
    if s[:5] == 'A267T' and len(s) == 6:
        return s[:5] + '0' + s[5]
    return s
adata.obs_names = [correct(name) for name in adata.obs_names]

In [ ]:
gene_scores = gene_scores.loc[adata.obs_names]
gene_scores = gene_scores[gene_scores.columns[isin(gene_scores.columns, adata.var_names)]]

In [ ]:
gene_scores.shape

In [ ]:
adata.X = np.log(adata.X.A + 1)

In [ ]:
adata[:,gene_scores.columns] = gene_scores

In [ ]:
sc.pp.filter_genes(adata, min_cells=1)
sc.pp.filter_cells(adata, min_genes=1)
adata

In [ ]:
# Loads the description files, for days
dct_day = {}
with open('../../data/sample_description/A172.sampleDescription.txt') as f_in:
    for line in f_in:
        line = line.split('|')
        dct_day[line[0]] = line[1].split('_')[1]
with open('../../data/sample_description/A257.sampleDescription.txt') as f_in:
    for line in f_in:
        line = line.split('|')
        dct_day[line[0]] = line[1].split('_')[1]
with open('../../data/sample_description/sample_annotation_comb.txt') as f_in:
    for line in f_in:
        line = line.split('\t')
        dct_day[line[0]] = 'day' + line[-1]    

In [ ]:
# Makes day nomenclature homogeneous

for i in range(1, 10):
    dct_day['A172T0%i' % i] = dct_day['A172T%i' % i]
    dct_day['A267T%i' % i] = dct_day['A267T0%i' % i]

col_day = array([dct_day.get(name, 'na') for name in adata.obs_names])
col_day[col_day == 'day0'] = 'day00'
col_day[col_day == 'DAY7'] = 'day07'
col_day[col_day == 'day7'] = 'day07'
col_day[col_day == 'day7\n'] = 'day07'
col_day[col_day == 'DAY7+2'] = 'day09'
col_day[col_day == 'day9\n'] = 'day09'
col_day[col_day == 'DAY7+3'] = 'day10'
col_day[col_day == 'day10\n'] = 'day10'
col_day[col_day == 'DAY7+4'] = 'day11'
col_day[col_day == 'day11\n'] = 'day11'
col_day[col_day == 'day14\n'] = 'day14'
col_day[col_day == 'day17\n'] = 'day17'
col_day[col_day == 'day22\n'] = 'day22'
adata = adata[col_day != 'na',:]
col_day = col_day[col_day != 'na']
adata.obs['day'] = col_day

# Signature scores computation

## EWSR1-FLI-1 related

In [ ]:
# Loads the ICs file
groups = {}
with open('../../data/ics_and_signatures.gmt') as f_in:
    for line in f_in:
        elems = line[:-1].split('\t')
        groups[elems[0]] = elems[2:]

In [ ]:
genes_ef1 = [
    g[:-3] for g in groups['EF1_DIRECT_CONFIRMED']
    if g[-3:] == '[1]'
]
genes_ef1 = [g for g in genes_ef1 if g in gene_scores.columns]
len(genes_ef1)

In [ ]:
to_exclude = ['KIF14', 'AURKA', 'AURKB']
most_var = np.var(gene_scores[genes_ef1], axis=0).sort_values()[:-3]
gene_scores_ef1 = gene_scores[most_var.index].T
n_ef1 = len(most_var)

In [ ]:
for day in sorted(set(adata.obs['day'])):
    c_names = adata.obs_names[adata.obs['day'] == day]
    gene_scores_ef1[day] = gene_scores_ef1[c_names].mean(1)

In [ ]:
figure(figsize=(20,25))
days = list(sorted(set(adata.obs['day'])))
for i, gene in enumerate(gene_scores_ef1.index):
    subplot(5,4,i+1)
    x = [gene_scores_ef1[adata.obs_names[adata.obs['day'] == day]].loc[gene] for day in days]
    boxplot(x)
    xticks(1 + arange(7), days)
    title(gene)

In [ ]:
ef1_genes = gene_scores_ef1.index[gene_scores_ef1['day22'] - gene_scores_ef1['day11'] > 1]
ef1_genes

In [ ]:
gene_scores['ic10plus'] = gene_scores[ef1_genes].mean(1)
adata.obs['ic10plus'] = gene_scores['ic10plus']

## Cell cycle related

In [ ]:
genes_g1s = [
    g for g in groups['G1_S_CHECKPOINT']
]
genes_g1s = [g for g in genes_g1s if g in gene_scores.columns]
most_var = np.var(gene_scores[genes_g1s], axis=0).sort_values()[:-3]
gene_scores_g1s = gene_scores[most_var.index].T
g1s_genes = most_var.index[-5:]
gene_scores['g1s'] = gene_scores[g1s_genes].mean(1)
adata.obs['g1s'] = gene_scores['g1s']

In [ ]:
genes_g2m = [
    g for g in groups['G2_M_CHECKPOINT']
]
genes_g2m = [g for g in genes_g2m if g in gene_scores.columns]
most_var = np.var(gene_scores[genes_g2m], axis=0).sort_values()[:-3]
gene_scores_g2m = gene_scores[most_var.index].T
g2m_genes = most_var.index[-5:]
gene_scores['g2m'] = gene_scores[g2m_genes].mean(1)
adata.obs['g2m'] = gene_scores['g2m']

In [ ]:
adata.obs['cellcycle'] = .5*adata.obs['g1s'] + .5*adata.obs['g2m']

## ICs

In [ ]:
# Sort genes by variance
ord_vars = adata.var_names[np.argsort(np.var(adata.X, axis=0))]

N = 2000 # N to keep
IC_genes = {
    gr: list(filter(lambda gene: gene in ord_vars[-N:], groups[gr]))
    for gr in filter(lambda s: s[:2] == 'IC', groups.keys())
}

In [ ]:
for gr in IC_genes:
    adata.obs[gr] = np.mean(adata[:,IC_genes[gr]].X, axis=1)

# RNA velocity

In [ ]:
# Preprocess adata for umap & velocity
sv.pp.filter_and_normalize(adata)
sv.pp.moments(adata)
sv.tl.velocity(adata, mode='stochastic')
sv.tl.velocity_graph(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sv.tl.velocity_embedding(adata, basis='umap')

In [ ]:
# just chooses a nice palette
adata.uns['day_colors'] = [
    "#03fdff",
    "#04cff4",
    "#00b4ef",
    "#0ca2ff",
    "#0960fe",
    "#0042f5",
    "#0603a9"
]

In [ ]:
# Custom colormap

from matplotlib.colors import LinearSegmentedColormap

cdict = {'red':   [[0.0,  0.2, 0.2],
                   [0.3,  0.2, 0.2],
                   [0.9,  1.0, 1.0],
                   [1.0,  1.0, 1.0]],
         'green': [[0.0,  0.2, 0.2],
                   [1.0,  0.2, 0.2]],
         'blue':  [[0.0,  1.0, 1.0],
                   [0.1,  1.0, 1.0],
                   [0.7,  0.2, 0.2],
                   [1.0,  0.2, 0.2]]}

cmp_rbg = LinearSegmentedColormap(name='my_RBG', segmentdata=cdict, N=256)

def plot_linearmap(cdict):
    newcmp = LinearSegmentedColormap('testCmap', segmentdata=cdict, N=256)
    rgba = newcmp(np.linspace(0, 1, 256))
    fig, ax = plt.subplots(figsize=(4, 3), constrained_layout=True)
    col = ['r', 'g', 'b']
    for xx in [0.25, 0.5, 0.75]:
        ax.axvline(xx, color='0.7', linestyle='--')
    for i in range(3):
        ax.plot(np.arange(256)/256, rgba[:, i], color=col[i])
    ax.set_xlabel('index')
    ax.set_ylabel('RGB')
    plt.show()
plot_linearmap(cdict)

In [ ]:
sv.settings.set_figure_params(dpi=150)
ax = sv.pl.velocity_embedding(adata, basis='umap', color='day', legend_loc='on data', alpha=.7, frameon=True, size=150,
                         title='', xlabel='', ylabel='', legend_fontsize=14, arrow_length=2, save='all_days.pdf',
                        edgecolors='black', linewidth=.3, show=True)

In [ ]:
sv.settings.set_figure_params(dpi=150)
sv.pl.velocity_embedding(adata, basis='umap', color='ic10plus', legend_loc='upper right', alpha=.9, frameon=True, size=150,
                         title='', xlabel='', ylabel='', legend_fontsize=14, arrow_length=2, save='ef1_score.pdf', color_map=cmp_rbg,
                        linewidth=.3)
sv.pl.velocity_embedding(adata, basis='umap', color='cellcycle', legend_loc='upper right', alpha=.9, frameon=True, size=150,
                         title='', xlabel='', ylabel='', legend_fontsize=14, arrow_length=2, save='cc_score.pdf', color_map=cmp_rbg,
                        linewidth=.3)

## ICs

In [ ]:
for gr in IC_genes:
    sv.pl.velocity_embedding(adata, basis='umap', color=gr, legend_loc='upper right', alpha=.9, frameon=True, size=150,
                         title='', xlabel='', ylabel='', legend_fontsize=14, arrow_length=2, save='cc_ic_%s.pdf' % gr, color_map=cmp_rbg,
                        linewidth=.3)

## Magnitude colouring

In [ ]:
adata.obs['vel_mag'] = np.sqrt(np.sum(adata.layers['velocity']**2, axis=1))

In [ ]:
sv.pl.velocity_embedding(adata, basis='umap', color='vel_mag', legend_loc='upper right', alpha=.9, frameon=True, size=150,
                         title='', xlabel='', ylabel='', legend_fontsize=14, arrow_length=2, save='vel_mag.pdf', color_map=cmp_rbg,
                        linewidth=.3)

In [ ]:
adata.layers['velocity'].shape